In [1]:
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans

In [2]:
# 1-1. 데이터 가져오기 

import pandas as pd 
import matplotlib.pyplot as plt 


socar = pd.read_csv('../../0. raw_data/insurance_fraud_detect_data.csv')
socar 

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,...,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,...,0,1,1,0,1,0,0,0,-1,0
15996,0,2,0,2,1,4,0,100,1,1,...,0,2,1,0,1,0,0,0,-1,0
15997,0,2,1,2,0,4,0,100,1,0,...,0,0,0,0,1,0,0,0,-1,1
15998,0,2,0,2,0,2,0,100,2,0,...,0,4,1,0,1,0,0,0,-1,0


In [3]:
# train-test 분리

df = socar[socar['test_set']==0]
df_test = socar[socar['test_set']==1]

In [4]:
# test_set 분리 
X_test = df_test.drop('fraud_YN', axis = 1)
y_test = df_test['fraud_YN']

In [5]:
# train-valid 분리 

from sklearn.model_selection import train_test_split

y_target = df['fraud_YN']
X_features = df.drop('fraud_YN', axis = 1)

X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size = 0.2, random_state = 13, stratify = y_target)

# 비율 확인 
print(y_train.value_counts()/y_train.shape[0]*100) 
print(y_valid.value_counts()/y_valid.shape[0]*100)

0    99.73794
1     0.26206
Name: fraud_YN, dtype: float64
0    99.728261
1     0.271739
Name: fraud_YN, dtype: float64


In [6]:
# k-means 를 사용한 속성 변경 
from sklearn.cluster import KMeans

kmeans10 = KMeans(10)
X_train_10 = kmeans10.fit_transform(X_train)
X_valid_10 = kmeans10.transform(X_valid)

In [7]:
# 학습 / 예측 / 평가 함수 

# 평가 함수 

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred):
    # confusion = confusion_matrix(y_test, pred) #오차행렬
    accuracy = accuracy_score(y_test,pred) # 정확도 
    precision = precision_score(y_test,pred) # 정밀도
    recall = recall_score(y_test,pred) # 재현율 
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred)
    return accuracy, precision, recall, f1, roc_auc

# 성능 DF 반환 

def get_result(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    return get_clf_eval(y_test, pred)

# 다수의 모델 성능 비교 

def get_result_df(models, model_names, X_train, y_train, X_test, y_test):
    col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc']
    tmp = []
    for model in models:
        tmp.append(get_result(model, X_train, y_train, X_test, y_test))

    return pd.DataFrame(tmp, columns=col_names, index=model_names)


# 성능 확인 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC


# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()


# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_10, y_train, X_valid_10, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996506,0.0,0.0,0.0,0.499611
RandomFore,0.996506,0.0,0.0,0.0,0.499611
LGBM,0.996118,0.0,0.0,0.0,0.499416
SVC,0.997283,0.0,0.0,0.0,0.500000


In [8]:
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, random_state=0).fit(socar)

In [9]:
socar['cluster'] =kmeans.labels_
socar.head()

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,...,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set,cluster
0,0,2,1,3,0,2,0,100,2,0,...,1,0,0,1,0,0,0,-1,0,0
1,0,1,0,4,0,4,0,0,3,1,...,1,1,1,2,1,1,1,0,1,0
2,0,1,0,4,1,4,0,100,3,1,...,3,1,0,2,1,1,1,0,1,1
3,0,3,1,3,0,4,0,0,2,0,...,3,1,0,1,0,0,0,-1,0,0
4,0,1,0,1,0,1,0,100,1,0,...,1,0,0,2,0,0,0,-1,0,0
